# Standard CNN Architectures
* https://keras.io/applications/

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
import matplotlib.pylab as plt
import numpy as np

In [8]:
from distutils.version import StrictVersion

In [9]:
import sklearn
print(sklearn.__version__)

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

0.19.0


In [10]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

1.2.1


In [11]:
import keras
print(keras.__version__)

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

2.0.8


In [12]:
import pandas as pd
print(pd.__version__)

assert StrictVersion(pd.__version__) >= StrictVersion('0.20.0')

0.20.3


## Preparation

In [23]:
# for VGG, ResNet, and MobileNet
INPUT_SHAPE = (224, 224)

# for InceptionV3, InceptionResNetV2, Xception
# INPUT_SHAPE = (299, 299)

In [24]:
EPOCHS = 10

In [13]:
!ls -lh

total 3.5G
-rwxrwxrwx 1 root root 303K Sep 27 17:02 Black_New_York_stuy_town_squirrel_amanda_ernlund.jpeg
-rwxrwxrwx 1 root root 140K Sep 27 15:54 cnn-augmentation.ipynb
-rwxrwxrwx 1 root root 1.6M Sep 27 15:54 cnn-comparing-all-models.ipynb
-rwxrwxrwx 1 root root 380K Sep 27 16:13 cnn-intro.ipynb
-rwxrwxrwx 1 root root 1.3M Sep 27 15:54 cnn-prediction.ipynb
-rwxrwxrwx 1 root root  36K Sep 27 17:11 cnn-standard-architectures.ipynb
-rwxrwxrwx 1 root root 198K Sep 27 16:21 cnn-train-augmented.ipynb
-rwxrwxrwx 1 root root  48K Sep 27 17:08 imagenet-pretrained.ipynb
-rwxrwxrwx 1 root root 495K Sep 27 17:02 london.jpg
drwxrwxrwx 0 root root  512 Sep 27 16:11 __MACOSX
-rwxrwxrwx 1 root root 127K Sep 27 17:01 Michigan-MSU-raschka.jpg
-rwxrwxrwx 1 root root 519K Sep 27 12:34 ml_intro.ipynb
-rwxrwxrwx 1 root root 113K Sep 27 15:54 nn-intro.ipynb
-rwxrwxrwx 1 root root   63 Sep 27 15:54 README.html
drwxrwxrwx 0 root root  512 Sep 27 16:11 speed-limit-signs
-rwxrwxrwx 1 root root 1.8M Sep 27 16:1

In [16]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, INPUT_SHAPE) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [17]:
# Load datasets.
ROOT_PATH = "./"
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
X, y = load_data(original_dir, type=".ppm")

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape

((303, 224, 224, 3), (303, 6))

## VGG

In [32]:
from keras import applications
# applications.VGG16?
# model = applications.VGG16(include_top=False, weights='imagenet')
model = applications.VGG16(classes=6, weights=None)

In [33]:
# lacks dropout as this only exists during training

# model.summary()

In [34]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
# %time model.fit(X_train, y_train, epochs=10, validation_split=0.3)

## MobileNet

In [19]:
from keras.applications.mobilenet import MobileNet

model = MobileNet(classes=6, weights=None)

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [21]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
# %time model.fit(X_train, y_train, epochs=10, validation_split=0.3)

Train on 212 samples, validate on 91 samples
Epoch 1/10
212/212 [==============================] - 150s - loss: 1.9871 - acc: 0.2217 - val_loss: 1.7920 - val_acc: 0.2088
Epoch 2/10
160/212 [=====================>........] - ETA: 31s - loss: 1.7673 - acc: 0.2625

KeyboardInterrupt: 

## ResNet

In [36]:
from keras.applications.resnet50 import ResNet50

model = ResNet50(classes=6, weights=None)

In [37]:
# model.summary()

In [38]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [39]:
%time model.fit(X_train, y_train, epochs=10, validation_split=0.3)

Train on 212 samples, validate on 91 samples
Epoch 1/10


KeyboardInterrupt: 